In [10]:
import pandas as pd

payments = pd.read_csv('payments.csv')
spent = pd.read_csv('spent.csv')

new_df = payments['ts'].str.split(' ', expand=True)
new_df.columns = ['ts_date','ts_time']
payments_df = pd.concat([new_df, payments], axis=1)

# превращаю в формат даты
payments_df['date'] = pd.to_datetime(payments_df['date'], format='%Y-%m-%d')
payments_df['ts_date'] = pd.to_datetime(payments_df['ts_date'], format='%Y-%m-%d')
# вычитаю из тс дату и фильтрую по 28 дням
payments_df['days'] = payments_df['ts_date'].sub(payments_df['date'], axis=0)
payments_df = payments_df.sort_values('days').drop(payments_df[payments_df.days != '28 days'].index)
# группирую по сумме дней

pay_pivot = payments_df.pivot_table(index='date', columns=['os'], values=['app_revenue'], aggfunc='sum')

# группирую по месяцам
payments = pay_pivot.groupby(pd.Grouper(freq='M')).sum()


# превращаю в формат даты
spent['date'] = pd.to_datetime(spent['date'], format='%Y-%m-%d')

# группирую по сумме дней
spent.groupby(['date']).sum()

# ////////

# Сводная таблица
spent_pivot = spent.pivot_table(index= 'date', columns = ['os'] , values= ['spend'] , aggfunc='sum')

# группирую по месяцам
spent = spent_pivot.groupby(pd.Grouper(freq='M')).sum()

# нахожу разницу между доходами и затратами
android = payments.app_revenue['android'].sub(spent.spend['android'])
ios = payments.app_revenue['ios'].sub(spent.spend['ios'])


# нахождение темпа роста
ios = ios[0]/ios[1]
android = android[0]/android[1]

android

0.5569738854597795